<center>
<h1>Exploratory Data Analysis</h1>
</center>

---

This is a first exploration of the data with the goal to gather some simple statistics and insights into the data.

In [ ]:
import os
import sys 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import re

sns.set_style('darkgrid')

sys.path.append("..")
from src.data import replace_mod

In [ ]:
data = pd.read_csv("../data/Peptides_and_iRT.tsv", sep=r'\t')
data.columns=["sequence_raw", "iRT"]
data.head(15)

In [ ]:
print(f"We have {len(data)} data points with iRTs ranging from {data.iRT.min()} to {data.iRT.max()}.")
print(f"The data has mean and standard deviation of {data.iRT.mean()} ± {data.iRT.std()}")
print(f"The central 99% quantile has a range of {np.round(data.iRT.quantile(0.005),3)} to {np.round(data.iRT.quantile(0.995), 3)}.")

In [ ]:
rows_duplicate = data.duplicated().sum()
print("Number of duplicate entries:", rows_duplicate)

In [ ]:
data.isna().mean().sort_values()

In [ ]:
plt.figure(figsize=(8,8))

data.hist(color="darkred",bins=45)

plt.xlabel("iRT")

plt.tight_layout()
plt.show()

## Derive some features from the peptide sequence

In [ ]:
data["sequence_length"] = data["sequence_raw"].apply(len)
data["is_mod"] = data["sequence_raw"].str.contains("\[").astype(int)

mod_index = data.query("is_mod == 1").index
re_mod = re.compile(r"\[[\+A-Za-z0-9]+\]")
data.loc[mod_index, "modification"] = data.query("is_mod == 1")["sequence_raw"].str.findall(re_mod)

In [ ]:
max_mod = data.loc[mod_index, "modification"].apply(len).max()
modification_types = data.query("is_mod == 1")["modification"].explode().unique()

print("We have up to {0:1d} modified AAs and {1:1d} types of modifications. We can use those as features.".format(max_mod, len(modification_types)))

print("Modifications:", modification_types)

In [ ]:
plt.figure()


amino_df = data["sequence_raw"].apply(lambda s: replace_mod(s, modification_types)).explode()
(amino_df.value_counts()).plot.bar(color='darkred', figsize=(8,8),rot=45)

plt.xlabel("Amino Acid")

plt.title("Amino acid frequencies")
plt.tight_layout()

plt.show()

In [ ]:
data["sequence_raw"].apply(lambda s: replace_mod(s, modification_types))

In [ ]:
plt.figure()

data.hist(color="darkred",bins=30, figsize=(8,8))

plt.tight_layout()
plt.show()

## See if there are any obvious correlations

In [ ]:
feat_correlations = data.fillna(-1).corr(method="pearson")

plt.figure(figsize=(10,10))
sns.heatmap(feat_correlations,
           square=True,
           center=0,
           annot=np.round(feat_correlations,3),
           fmt="",
           linewidths=.5,
           cmap="vlag",
           cbar_kws={"shrink": 0.8},)

plt.tight_layout()
plt.show()

In [ ]:
data["iRT_normed"] = (data["iRT"] - data["iRT"].mean()) / (data["iRT"].std()) 

In [ ]:
data["iRT_scaled"] = (data["iRT"] - data["iRT"].min()) / (data["iRT"].max() - data["iRT"].min()) 

## Conclusions

A first glance at the data has revealed some simple correlations between the sequence length and the iRT, but more feature engineering seems to be appropriate.

Since the iRT can be negative, there are no obvious faulty data points (duplicates, missing values). Only a few outliers with large and negative iRT are in the dataset. We might want to remove them later for training ML models.

Modifications of the AAs occur only on M[+O] and C[+C2+H3+N+O], the latter of which only occurs in a modified state.


### Code export:

The results of this notebook have been written to `src/data/data_ingestion.py`